In [1]:
# importa os pacotes necessários
import numpy as np
import os, gc, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score

from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('kaggle/data_pca.csv')
df_labels = pd.read_csv('kaggle/data_labels.csv')
df.shape,df_labels.shape

((25000, 2500), (25000, 1))

In [4]:
X_train = df.iloc[:20556]
X_test  = df.iloc[20556:]
y_train = df_labels.iloc[:20556]
y_test  = df_labels.iloc[20556:]
X_train.shape,X_test.shape

((20556, 2500), (4444, 2500))

# Logistic Regression
Utilizando regressão logistica para ter uma base de comparação com os demais modelos.

In [5]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train, y_train)

CPU times: user 1min 2s, sys: 2.42 ms, total: 1min 2s
Wall time: 1min 3s


In [6]:
# avalia predicao
y_pred = logreg.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.715465,0.710688,0.713096,0.713076,0.713076
precision,0.709606,0.716705,0.713096,0.713155,0.713155
recall,0.721422,0.704770,0.713096,0.713096,0.713096
support,2222.000000,2222.000000,0.713096,4444.000000,4444.000000


Somente os 1000 primeiros componentes.

In [9]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train.iloc[:,:1000], y_train)

CPU times: user 6.08 s, sys: 37 µs, total: 6.08 s
Wall time: 6.18 s


In [10]:
# avalia predicao
y_pred = logreg.predict(X_test.iloc[:,:1000])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.733875,0.735070,0.734473,0.734472,0.734472
precision,0.735533,0.733423,0.734473,0.734478,0.734478
recall,0.732223,0.736724,0.734473,0.734473,0.734473
support,2222.000000,2222.000000,0.734473,4444.000000,4444.000000


# SVM